# Get GPTapi secret key

In [141]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# Crawling

In [142]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

#Input URL
url = "https://www.fendi.com/kr-ko/%EC%97%AC%EC%84%B1/%EA%B0%80%EB%B0%A9_1/%EB%9D%BC%EC%A7%80-%EB%A1%A4-%ED%8E%98%ED%80%B8-%EC%8A%A4%ED%8A%B8%EB%9D%BC%EC%9D%B4%ED%94%84%EC%99%80-%EC%98%90%EB%A1%9C%EC%9A%B0-ff-%ED%8C%A8%EB%B8%8C%EB%A6%AD-%EB%A6%AC%EB%B2%84%EC%84%9C%EB%B8%94-%EC%87%BC%ED%8D%BC-8bh425as86f1pio"
driver.get(url)

#js load 될 떄 까지 delay
script = """return document.readyState === 'complete';"""
while not driver.execute_script(script):
    pass

html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

with open("test.html", "w", encoding="utf-8") as file:
    file.write(str(soup.prettify()))

driver.quit()

# Initalize GPT Assistants

In [143]:
from openai import OpenAI
 
client = OpenAI()
assistant = client.beta.assistants.create(
  name="Crawling Assistants",
  instructions="You are a world-class algorithm for extracting imformation in html file. Extract the product option name from this html file.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [144]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Crwalier")
 
# Ready the files for upload to OpenAI
file_paths = ["test.html"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [145]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [146]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("test.html", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Extract the product options from this html file. you must answer only product options list, and option name",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_i7T0yHesJH2IfGTQlBAEEfoQ'])


In [147]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
client = OpenAI()
 
class EventHandler(AssistantEventHandler):
    '''
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)
    '''
    
    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Extract the product options from this html file. ",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

- Color options: 옐로우, 라이트 블루, 베이지, 블루
- Size options: TU
- Quantity options: 1, 2

